# 強化学習のmelowmaxについて

今日はこの[論文](https://arxiv.org/abs/1612.05628)について説明していきます．

強化学習の更新式は次のように書かれます．

$$
Q(s,a) \leftarrow R(s,a) + \gamma \sum_{\hat{s} \in S} P(s,a,\hat{s})\bigotimes_{\hat{a}} Q(\hat{s},\hat{a})
$$

ここで, $Q$ は状態行動価値関数で, $R$ は報酬関数, $\gamma$ は $(0,1]$ で $P$ は確率遷移関数とします.

ここで問題となってくるのは, $\bigotimes$ にどんな作用素を持ってくるのかです．

見慣れている形(Q学習など)は $\max$ をとっていますね．

ここで色々な作用素の特徴を上の論文に準じて述べていきます．

---

ここからは $\mathbf X = x_1 , ..., x_n$ とします．

### $\max$ 作用素について

$$
\max{\mathbf X} = \max_{i \in (x_1,...,x_n)}x_i
$$

この作用素のメリットとしてはnon-expansionということになります．つまり, ある不動店に収束すると保証されます．

デメリットは微分ができない, 最大化する部分以外の要素を無視していることです．

しかしながら, 実装の容易さやシンプルであることから重宝されていますね．


---

### mean 作用素について

$$
mean(\mathbf{X}) = \frac{1}{n} \sum_{i=1}^{n} x_i
$$

この作用素はメリットとしてはnon-expantionでありながら, 微分可能です．

しかしながら, 価値が高いものと低いものの判断が平均によって評価しずらくなってしまいます．



---

### ボルツマン作用素について

$$
boltz_{\beta}(\mathbf{X}) = \frac{\sum_{i=1}^{n} x_i e^{\beta x_i}}{\sum_{i=1}^{n}e^{\beta x_i}}
$$

この作用素は微分可能で, 価値が高いものと低いものの区別もしてくれるようになっていますが, non-expantionでないとが知られています．

ちなみに $\beta \rightarrow \infty$ で $\max$ 作用素, $\beta \rightarrow 0$ で　$mean$ 作用素と同じ働きをします．

---

上の論文ではこのボルツマン作用素に似ているものを考えて, かつnon-expantionである作用素を提案することを目的としています．

それが $mellowmax$ です．

まず式を確認します．

---

### $mellowmax$ 作用素について

$$
mm_\omega\mathbf(X) = \frac{log(\frac{1}{n} \sum_{i=1}^{n} e^{\omega x_i})}{\omega}
$$

上の作用素はnon-expantionであることが論文中で示されています．